In [1]:
import pandas as pd
import datetime
from sqlalchemy import create_engine

In [2]:
data = pd.read_csv("Data/data.csv")
genre_data = pd.read_csv("Data/data_w_genres.csv")
year_data = pd.read_csv("Data/data_by_year.csv")

In [3]:

organized_data = data[["id",
                       "name",
                       "artists",
                       "year",
                       "duration_ms",
                       "explicit",
                       "acousticness",
                       "danceability",
                       "energy",
                       "instrumentalness",
                       "liveness",
                       "loudness",
                       "popularity",
                       "speechiness",
                       "tempo"                     
                      ]]

In [4]:
spotify_data = organized_data.rename(columns={"id": "ID", 
                                              "name": "Song", 
                                              "artists": "Artists", 
                                              "year": "Year",
                                              "duration_ms": "Duration (M:S)",
                                              "explicit": "Explicit",
                                              "acousticness": "Acousticness",
                                              "danceability": "Danceability",
                                              "energy": "Energy",
                                              "instrumentalness": "Instrumentalness",
                                              "liveness": "Liveness",
                                              "loudness": "Loudness", 
                                              "popularity": "Popularity", 
                                              "speechiness": "Speechiness", 
                                              "tempo": "Tempo"})

In [5]:
#convert 'Duration' column in milliseconds to "Minutes:Seconds" format 
millis = spotify_data["Duration (M:S)"]


MS_list=[]
for m in millis:
   
   
    date = datetime.datetime.fromtimestamp(m/1000.0)
    date = date.strftime('%M:%S')
    MS_list.append(date)
    
MS_list   

spotify_data["Duration (M:S)"]=MS_list

In [6]:
#clean the 'Artist' column
artists = spotify_data['Artists']
artists_l = []
for a in artists:
    test1 = a.strip('[]\'\'').replace("'",'')
    artists_l.append(test1)
spotify_data['Artists'] = artists_l

In [15]:
#round the columns to the nearest 2 digit decimal point
spotify_final=spotify_data.round({'Acousticness':2,'Danceability':2,'Energy':2,'Instrumentalness':2,'Liveness':2,'Loudness':2,'Speechiness': 2, 'Tempo': 2})
spotify_final

,ID,Song,Artists,Year,Duration (M:S),Explicit,Acousticness,Danceability,Energy,Instrumentalness,Liveness,Loudness,Popularity,Speechiness,Tempo
0,6KbQ3uYMLKb5jDxLF7wYDD,Singende Bataillone 1. Teil,Carl Woitschach,1928,02:38,0,1.00,0.71,0.20,0.56,0.15,-12.43,0,0.05,118.47
1,6KuQTIu1KoTTkLXKrwlLPV,"Fantasiestücke, Op. 111: Più tosto lento","Robert Schumann, Vladimir Horowitz",1928,04:42,0,0.99,0.38,0.01,0.90,0.08,-28.45,0,0.05,83.97
2,6L63VW0PibdM1HDSBoqnoM,Chapter 1.18 - Zamek kaniowski,Seweryn Goszczyński,1928,01:44,0,0.60,0.75,0.22,0.00,0.12,-19.92,0,0.93,107.18
3,6M94FkXd15sOAOQYRnWPN8,Bebamos Juntos - Instrumental (Remasterizado),Francisco Canaro,1928,03:00,0,1.00,0.78,0.13,0.89,0.11,-14.73,0,0.09,108.00
4,6N6tiFZ9vLTSOIxkj8qKrd,"Polonaise-Fantaisie in A-Flat Major, Op. 61","Frédéric Chopin, Vladimir Horowitz",1928,11:27,0,0.99,0.21,0.20,0.91,0.10,-16.83,1,0.04,62.15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
169904,4KppkflX7I3vJQk7urOJaS,Skechers (feat. Tyga) - Remix,"DripReport, Tyga",2020,02:43,1,0.17,0.88,0.44,0.00,0.09,-7.46,75,0.14,100.01
169905,1ehhGlTvjtHo2e4xJFB0SZ,Sweeter (feat. Terrace Martin),"Leon Bridges, Terrace Martin",2020,02:47,0,0.02,0.72,0.38,0.03,0.11,-10.91,64,0.04,128.00
169906,52eycxprLhK3lPcRLbQiVk,How Would I Know,"Kygo, Oh Wonder",2020,03:00,0,0.54,0.51,0.54,0.00,0.11,-9.33,70,0.10,123.70
169907,3wYOGJYD31sLRmBgCvWxa4,I Found You,"Cash Cash, Andy Grammer",2020,02:47,0,0.07,0.65,0.76,0.00,0.22,-2.56,70,0.04,129.92


In [18]:
#write our cleaned data to 'Data/SpotifyData1.csv'
spotify_final.to_csv("Data/SpotifyData1.csv", index=False, header=True)

In [16]:
engine = create_engine('sqlite:///Spotify.db', echo=True)
sqlite_connection = engine.connect()

2020-09-02 09:44:06,054 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2020-09-02 09:44:06,060 INFO sqlalchemy.engine.base.Engine ()
2020-09-02 09:44:06,062 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2020-09-02 09:44:06,063 INFO sqlalchemy.engine.base.Engine ()


In [17]:
spotify_table = "Spotify_Data"
spotify_final.to_sql(spotify_table, sqlite_connection, if_exists='replace')

2020-09-02 09:44:10,883 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("Spotify_Data")
2020-09-02 09:44:10,885 INFO sqlalchemy.engine.base.Engine ()
2020-09-02 09:44:10,887 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("Spotify_Data")
2020-09-02 09:44:10,889 INFO sqlalchemy.engine.base.Engine ()
2020-09-02 09:44:10,891 INFO sqlalchemy.engine.base.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2020-09-02 09:44:10,894 INFO sqlalchemy.engine.base.Engine ()
2020-09-02 09:44:10,897 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("Spotify_Data")
2020-09-02 09:44:10,900 INFO sqlalchemy.engine.base.Engine ()
2020-09-02 09:44:10,904 INFO sqlalchemy.engine.base.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = 'Spotify_Data' AND type = 'table'
2020-09-02 09:44:10,905 INFO sqlalchemy.engine.base.Engine ()
2020-09-02 09:44:10,909 INFO sqlalchemy.engine.base.Engine PRAGMA main.fore

In [27]:
Danceability=spotify_final[['ID','Song','Artists','Year','Duration (M:S)','Danceability']]
Danceability_order=Danceability.sort_values('Danceability',inplace=False,ascending=False)


,ID,Song,Artists,Year,Duration (M:S),Danceability
134698,1MgM0WDaoQ2A3bnIQTR6ty,Go Girl,"Pitbull, Trina, Young Bo",2012,03:49,0.99
61920,5IbMUI4rcWtXlq83UWqTcQ,Go Girl,"Young Boss, Trina, Pitbull",2007,03:49,0.99
84909,5YIF6HSOtHN9HdcE5IPzMe,Funky Cold Medina,Tone-Loc,1989,04:08,0.99
37596,1JhFQ8L6QZxQchBa5qJrxS,Barney Theme Song,Barney,1994,00:56,0.98
77683,5UJ3N25ukIruG4Do8RDHra,UP LIKE AN INSOMNIAC - Freestyle,XXXTENTACION,2017,02:31,0.98
74409,2CwGWqDUOImtIn1vUl4i3r,Wake Up,Run–D.M.C.,1984,05:31,0.98
86811,3XVozq1aeqsJwpXrEZrDJ9,Ice Ice Baby,Vanilla Ice,2008,04:14,0.98
12974,17DML8FAjSvGbEPC4fm6yW,Black Is Black,Jungle Brothers,1988,03:37,0.98
67620,7MwGWKdDGeop9D8bZN37hc,Binibi Rocha - Live,Andrew E.,1990,03:39,0.98
105994,0wbDgMuAoy7O7pL3a69uZx,Give It To Me,"Timbaland, Justin Timberlake, Nelly Furtado",2007,03:54,0.98


In [28]:
spotify_table = "Groovy_Songs"
Danceability_order.to_sql(spotify_table, sqlite_connection, if_exists='replace')


2020-09-02 19:34:35,474 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("Groovy_Songs")
2020-09-02 19:34:35,475 INFO sqlalchemy.engine.base.Engine ()
2020-09-02 19:34:35,477 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("Groovy_Songs")
2020-09-02 19:34:35,478 INFO sqlalchemy.engine.base.Engine ()
2020-09-02 19:34:35,481 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE "Groovy_Songs" (
	"index" BIGINT, 
	"ID" TEXT, 
	"Song" TEXT, 
	"Artists" TEXT, 
	"Year" BIGINT, 
	"Duration (M:S)" TEXT, 
	"Danceability" FLOAT
)


2020-09-02 19:34:35,481 INFO sqlalchemy.engine.base.Engine ()
2020-09-02 19:34:35,497 INFO sqlalchemy.engine.base.Engine COMMIT
2020-09-02 19:34:35,500 INFO sqlalchemy.engine.base.Engine CREATE INDEX "ix_Groovy_Songs_index" ON "Groovy_Songs" ("index")
2020-09-02 19:34:35,503 INFO sqlalchemy.engine.base.Engine ()
2020-09-02 19:34:35,513 INFO sqlalchemy.engine.base.Engine COMMIT
2020-09-02 19:34:35,688 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
20

In [31]:
Spa_music0=spotify_final[['ID','Song','Artists','Year','Duration (M:S)','Instrumentalness']]
Spa_music=Spa_music0.sort_values('Instrumentalness',inplace=False,ascending=False)
Spa_music 

,ID,Song,Artists,Year,Duration (M:S),Instrumentalness
15930,4fD4bjo6ZQp8tlF8Kyy6Ad,Rain Sound: Pouring,Rain Sounds XLE Library,2017,02:21,1.0
46972,605JTCltCe1IcqrGXQ294K,Brown Noise for Sleep,Sounds for Life,2009,11:10,1.0
143967,5svhO7n5a0IWF13hqZWaVc,City Rain,Rain Recorders,2017,04:00,1.0
125835,50N6EPhnmazJFkWvzqynhb,Downpoar,Rain Sound Studio,2017,02:32,1.0
23862,2eWqBNH136U8Et0UTxBacB,White Noise - BP 228 hz,Granular,2017,01:44,1.0
...,...,...,...,...,...,...
69857,6wIBZCIbbZh6ACDkWC5TX7,Ah Yeah,"Robert Glasper Experiment, Musiq Soulchild, Ch...",2012,05:08,0.0
69856,6ol1tnuiEhigHqHPwqQxeq,Stoner Girl (feat. Pat Brown),"MOD SUN, Pat Brown",2012,04:09,0.0
69855,4vpNOBwJSl8HA3pLXY6c8U,Druggys Wit Hoes Again (feat. Ab Soul),"ScHoolboy Q, Ab-Soul",2012,03:39,0.0
69854,1zugfOhn6BebboqGcpUlba,We Are Young (Glee Cast Version),Glee Cast,2012,04:09,0.0


In [32]:
spotify_table = "Spa_music"
Spa_music.to_sql(spotify_table, sqlite_connection, if_exists='replace')

2020-09-02 20:01:04,735 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("Spa_music")
2020-09-02 20:01:04,737 INFO sqlalchemy.engine.base.Engine ()
2020-09-02 20:01:04,738 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("Spa_music")
2020-09-02 20:01:04,739 INFO sqlalchemy.engine.base.Engine ()
2020-09-02 20:01:04,741 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE "Spa_music" (
	"index" BIGINT, 
	"ID" TEXT, 
	"Song" TEXT, 
	"Artists" TEXT, 
	"Year" BIGINT, 
	"Duration (M:S)" TEXT, 
	"Instrumentalness" FLOAT
)


2020-09-02 20:01:04,742 INFO sqlalchemy.engine.base.Engine ()
2020-09-02 20:01:04,749 INFO sqlalchemy.engine.base.Engine COMMIT
2020-09-02 20:01:04,750 INFO sqlalchemy.engine.base.Engine CREATE INDEX "ix_Spa_music_index" ON "Spa_music" ("index")
2020-09-02 20:01:04,751 INFO sqlalchemy.engine.base.Engine ()
2020-09-02 20:01:04,755 INFO sqlalchemy.engine.base.Engine COMMIT
2020-09-02 20:01:04,851 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-09-02 20